In [1]:
import numpy as np
import scipy.io as sio
import os
import euGenFuncs as eu
import matplotlib.pyplot as plt
%matplotlib inline
import imp 

In [2]:
subs=[264, 273, 862, 1125]
pth='/Users/davidgroppe/PycharmProjects/SZR_ANT/MODELS/svmAesFinale_1'
n_subs=len(subs)
try_thresh=np.arange(0.2,0.7,0.1)
n_thresh=len(try_thresh)

#preallocate mem
sens=np.zeros((n_subs, n_thresh))
total_hrs=np.zeros(n_subs)
total_clin_szrs=np.zeros(n_subs)
fp_per_day=np.zeros((n_subs, n_thresh))
mn_stim_lat=np.zeros((n_subs, n_thresh))
stim_lat_list=list()
stim_hit_list=list()
# Loop over files
for thresh_ct, thresh in enumerate(try_thresh):
    for sub_ct, sub in enumerate(subs):
        in_fname=str(sub)+'_thresh_0-'+str(int(10*thresh))+'_refract_0-5_stim_results.npz'
        npz=np.load(os.path.join(pth,in_fname))
        sens[sub_ct,thresh_ct]=npz['sens']
        fp_per_day[sub_ct,thresh_ct]=npz['fp_per_hour']*24
        mn_stim_lat[sub_ct,thresh_ct]=np.mean(npz['stim_latency_sec'])
        if thresh_ct==0:
            total_hrs[sub_ct]=npz['total_hrs']
            total_clin_szrs[sub_ct]=npz['n_clin_szr']
        elif thresh_ct==3:
            stim_lat_list.append(npz['stim_latency_sec'])
            stim_hit_list.append(npz['stim_latency_hit'])
            print(sub)
            print(npz['stim_latency_sec'])
            print(npz['stim_latency_hit'])
            print('stim_latency_hit %d' % np.sum(npz['stim_latency_hit']))
            print('n_true_pos %d' % npz['n_true_pos'])
            print('sens %f' % npz['sens'])
print('npz keys are {}'.format(npz.keys()))
print('Mean SD sensitivity: %f (%f)' % (np.mean(sens),np.std(sens)))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/davidgroppe/PycharmProjects/SZR_ANT/MODELS/svmAesFinale_1/264_thresh_0-2_refract_0-5_stim_results.npz'

In [ ]:
print(npz['stim_latency_sec'])
print(npz['stim_latency_hit'])
print(npz['refract_min'])
26555-25378

In [ ]:
temp_fname='/Users/davidgroppe/Desktop/26402102_0036_yhat.npz'
sis=np.load(temp_fname)
sis.keys()
temp_fname='/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_GENERAL/1125_y_112500102_0025.mat'
mat=sio.loadmat(temp_fname)
print(mat.keys())
print(mat['se_szr_class'].shape)
print(mat['se_time_sec'].shape)
print(mat['szr_class_dec'].shape)
print(mat['time_dec'].shape)

In [ ]:
# Report total hours of EEG & # of clinical szrs
print('Total hours of test data: %f' % np.sum(total_hrs))
print('Total days of test data: %f' % np.sum(total_hrs/24))
print('Mean (SD) days of test data: %f (%f)' % (np.mean(total_hrs/24),np.std(total_hrs/24)))
print('Total # of clinical szrs: %d' % np.sum(total_clin_szrs))
print('Mean (SD) # of clinical szrs: %f (%f)' % (np.mean(total_clin_szrs),np.std(total_clin_szrs)))

In [ ]:
# Plot Sensitivity
plt.figure(1)
plt.clf()
plt.plot(try_thresh,np.mean(sens,axis=0),'ro-')
for sub_ct in range(n_subs):
    plt.plot(try_thresh+(np.random.rand(1)-.5)/50,sens[sub_ct,:],'.')
plt.xlabel('Threshold')
plt.ylabel('Pptn of Szrs Stimulated')

thresh_id=3
print('For Thresh: %f' % try_thresh[thresh_id])
print('Mean (SD) Sensitivity: %f (%f)' % (np.mean(sens[:,thresh_id]),np.std(sens[:,thresh_id])))
for sub_ct in range(n_subs):
    print('%d: %f' % (subs[sub_ct],sens[sub_ct,thresh_id]))

In [ ]:
# Plot False Positive Stimulations/Day
plt.figure(2)
plt.clf()
plt.plot(try_thresh,np.mean(fp_per_day,axis=0),'ro-')
for sub_ct in range(n_subs):
    plt.plot(try_thresh+(np.random.rand(1)-.5)/50,fp_per_day[sub_ct,:],'.')
plt.xlabel('Threshold')
plt.ylabel('False postitive stim/day')
print('For Thresh: %f' % try_thresh[thresh_id])
print('Mean (SD) False Positive Stim/Day: %f (%f)' % (np.mean(fp_per_day[:,thresh_id]),np.std(fp_per_day[:,thresh_id])))

In [ ]:
# Plot Mean Stim Latency for each Clinical Szr
plt.figure(3)
plt.clf()
plt.plot(try_thresh,np.mean(mn_stim_lat,axis=0),'ro-')
for sub_ct in range(n_subs):
    plt.plot(try_thresh+(np.random.rand(1)-.5)/50,mn_stim_lat[sub_ct,:],'.')
plt.xlabel('Threshold')
plt.ylabel('Seconds to Onset')
print('For Thresh: %f' % try_thresh[thresh_id])
print('Mean (SD) False Positive Stim/Day: %f (%f)' % (np.mean(mn_stim_lat[:,thresh_id]),
                                                      np.std(mn_stim_lat[:,thresh_id])))

In [ ]:
# Boxplot-ish of stimulation latencies
len(stim_lat_list)
plt.figure(4)
for sub_ct in range(n_subs):
    stim_lat_temp=stim_lat_list[sub_ct][stim_hit_list[sub_ct]==1]
    #stim_lat_temp=stim_lat_list[sub_ct][np.abs(stim_lat_list[sub_ct])<100]
    
    # Plot individual szr stim latencies
    for stim_lat in stim_lat_temp:
    #for stim_lat in stim_lat_list[sub_ct]:
        plt.plot(1+sub_ct+(np.random.rand(1)-0.5)/3,stim_lat,'b.',markersize=8)
    avg_lat=np.mean(stim_lat_temp)
    #avg_lat=np.median(stim_lat_temp)
    plt.plot(1+np.asarray([-.2, 0.2])+sub_ct,[avg_lat, avg_lat],'r-',linewidth=3)
    sd_lat=np.std(stim_lat_temp)
    plt.plot([1+sub_ct, 1+sub_ct],[avg_lat-sd_lat, avg_lat+sd_lat],
             color=(93/255,49/255,168/255),linewidth=3)
    
# xlim=plt.xlim()
xlim=[0.5, 4.5]
plt.ylim([-20, 25])
plt.plot(xlim,[0, 0],'k--')
plt.xlim(xlim)
plt.xticks(np.arange(1,5))
plt.tick_params(labelsize=20)
# plt.ylabel('Seconds',fontsize=28)
# plt.xlabel('Patient',fontsize=28)
plt.savefig('aes_stim_latency.pdf')
print('done!')

In [ ]:
plt.tick_params?

In [ ]:
print('Max possible stims/day (given refractory period): %d' % (2*60*24))